# Introduction

I explored the factors influencing salaries in the data analytics field using a dataset from Kaggle which was Data Science Job Posting on Glassdoor for reference and skill practice, the dataset was set in the context of the United States. Throughout the process, I utilized Python to clean data. I overlooked the integrity check since I used the dataset solely for reference. After a quick review of the dataset, it was observed that it could be leveraged to explore factors such as company size, location, job titles, and the industries involved, such as healthcare and construction.

### Step 1

The first step was to import the necessary libraries and modules for the cleaning process. Then, I downloaded the dataset using the Kaggle API. Once I had the data, I loaded it using pandas and conducted an overview to gain a general understanding of the dataset, removing any basic errors.

In [4]:
import pandas as pd 
import kaggle as kg
import zipfile as zf
import re

In [5]:
# download dataset from kaggle using the Kaggle API
!kaggle datasets download -d rashikrahmanpritom/data-science-job-posting-on-glassdoor

Dataset URL: https://www.kaggle.com/datasets/rashikrahmanpritom/data-science-job-posting-on-glassdoor
License(s): CC0-1.0
data-science-job-posting-on-glassdoor.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
# extract the file from the downloaded zip file
zipfile_name = 'data-science-job-posting-on-glassdoor.zip'
with zf.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [7]:
df = pd.read_csv(r"Uncleaned_DS_jobs.csv")
df.set_index('index', inplace=True)

In [8]:
#To see the total size of the table
df.shape

(672, 14)

In [9]:
#To see whether the values in rows and cols fit with the size of the table as well as the data types of each column
df.info() 

<class 'pandas.core.frame.DataFrame'>
Index: 672 entries, 0 to 671
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          672 non-null    object 
 1   Salary Estimate    672 non-null    object 
 2   Job Description    672 non-null    object 
 3   Rating             672 non-null    float64
 4   Company Name       672 non-null    object 
 5   Location           672 non-null    object 
 6   Headquarters       672 non-null    object 
 7   Size               672 non-null    object 
 8   Founded            672 non-null    int64  
 9   Type of ownership  672 non-null    object 
 10  Industry           672 non-null    object 
 11  Sector             672 non-null    object 
 12  Revenue            672 non-null    object 
 13  Competitors        672 non-null    object 
dtypes: float64(1), int64(1), object(12)
memory usage: 78.8+ KB


In [10]:
# Find indices of duplicate rows for checking
check_duplicate = df.index[df.duplicated()].tolist()
print(check_duplicate)

[135, 136, 358, 359, 360, 361, 362, 389, 496, 497, 498, 499, 500]


In [11]:
#drop all the duplicates after making sure that it would have a bad affect on the analysis
df = df.drop_duplicates() 

### Step 2

After reviewing the table, I noticed that the format of the Salary Estimate column was not suitable for use. Therefore, I decided to split this column into two: Minimum Salary (Min Sal) and Maximum Salary (Max Sal).

In [12]:
#only take the salary range  
df['Salary Estimate'] = df['Salary Estimate'].str.split('(').str[0]

#split Salary Estimate in to 2 cols Min Sal and Max Sal
df[['Min Sal','Max Sal']] = df['Salary Estimate'].str.split('-',expand=True)

# Remove 'Min Sal', 'Max Sal' and store it
min_sal = df.pop('Min Sal')  
max_sal = df.pop('Max Sal') 
 # Insert 'Min Sal' and 'Max Sal' at the third position which is after 'Salary Estimate'
df.insert(2, 'Min Sal', min_sal) 
df.insert(3, 'Max Sal', max_sal)  

#Replace everything except digits and change data type of 'Min Sal' and 'Max Sal' into int
df['Min Sal'] = df['Min Sal'].apply(lambda x: int(re.sub(r'\D', '', str(x)))* 1000) 
df['Max Sal'] = df['Max Sal'].apply(lambda x: int(re.sub(r'\D', '', str(x)))* 1000) 

pd.set_option('display.max_rows', 10)
df #Let's have a look at it

,Job Title,Salary Estimate,Min Sal,Max Sal,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
index,,,,,,,,,,,,,,,,
0,Sr Data Scientist,$137K-$171K,137000,171000,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,$137K-$171K,137000,171000,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,$137K-$171K,137000,171000,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$137K-$171K,137000,171000,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,$137K-$171K,137000,171000,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,Data Scientist,$105K-$167K,105000,167000,Summary\n\nWe’re looking for a data scientist ...,3.6,TRANZACT\n3.6,"Fort Lee, NJ","Fort Lee, NJ",1001 to 5000 employees,1989,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
668,Data Scientist,$105K-$167K,105000,167000,Job Description\nBecome a thought leader withi...,-1.0,JKGT,"San Francisco, CA",-1,-1,-1,-1,-1,-1,-1,-1
669,Data Scientist,$105K-$167K,105000,167000,Join a thriving company that is changing the w...,-1.0,AccessHope,"Irwindale, CA",-1,-1,-1,-1,-1,-1,-1,-1


There were too many rows for me to manually review, so I decided to check if any salary values were unusually low or high compared to the majority.

In [13]:
#check for under 5 or 6-digit numbers
for x in ['Min Sal', 'Max Sal']:
    for y in range(df.shape[0]):
        value_length = len(str(df[x].iloc[y]))  
        if value_length < 5 or value_length > 6: 
            print(f'Col {x} Row {y} : {df[x].iloc[y]}')

print("There's no value over 6 digits or less than 5 digits.")

There's no value over 6 digits or less than 5 digits.


And it turned out there were no unusually low or high salaries compared to the majority.

### Step 3

After using the groupby function, I noticed there were too many job titles. However, I realized they could generally be grouped into six main categories: Analytics Manager, Data Analyst, Data Architect, Data Modeler, Data Scientist, and Machine Engineer.

In [14]:
df['Job Title'] = df['Job Title'].apply(lambda x: 'Data Scientist' if 'data' and 'scientist' in str(x).lower() else x)
df['Job Title'] = df['Job Title'].apply(lambda x: 'Data Analyst' if 'data' and 'analyst' in str(x).lower() else x)
df['Job Title'] = df['Job Title'].apply(lambda x: 'Data Engineer' if 'data' and 'engineer' in str(x).lower() else x)
df['Job Title'] = df['Job Title'].apply(lambda x: 'Data Modeler' if 'data' and 'modeler' in str(x).lower() else x)
df['Job Title'] = df['Job Title'].apply(lambda x: 'Machine Engineer' if 'machine' and 'engineer' in str(x).lower() else x)
df['Job Title'] = df['Job Title'].apply(lambda x: 'Data Science' if 'data' and 'science' in str(x).lower() else x)
df['Job Title'] = df['Job Title'].apply(lambda x: 'Data Scientist' if 'data' and 'science' in str(x).lower() else x)
df['Job Title'] = df['Job Title'].apply(lambda x: 'Analytics Manager' if 'analytics' and 'manager' in str(x).lower() else x)
df['Job Title'] = df['Job Title'].apply(lambda x: 'Analytics Manager' if 'data' and 'management' in str(x).lower() else x)

df.groupby('Job Title').mean(numeric_only=True).round(2)


,Min Sal,Max Sal,Rating,Founded
Job Title,,,,
Analytics Manager,104750.00,122250.00,4.05,1993.00
Data Analyst,92290.91,139145.45,3.76,1981.47
Data Architect,128000.00,201000.00,4.20,1996.00
Data Modeler,98500.00,126250.00,3.55,1881.50
Data Scientist,100325.63,151346.82,3.52,1586.47
Machine Engineer,94723.68,137381.58,3.96,1910.63


### Step 4

I completed resolving the job title and salary range issues. I then moved on to the "Size" and "Industry" column, which was another factor influencing salaries. I planned to use the groupby function to explore the values present in this column.

I noticed that companies were categorized into seven different sizes, but there were two values that were unusable: -1 and "Unknown". Along with that there were fifty seven industries and one value -1. Therefore, I decided to drop all rows that contained the those values.

In [15]:
#List out all the unusables
condition = df.index[df['Size'] == '-1'].tolist()
condition_2 = df.index[df['Size'].str.contains('Unknown')].tolist()
condition_3 = df.index[df['Industry'] == '-1'].tolist()

#take out all indexes distinctively, make it can cause some error if it is not distinctively selected
final_condition = list(set(condition) | set(condition_2) | set(condition_3))

#loop through and drop all the unusable values
for x in final_condition:
    df = df.drop(index=x)

### Step 5

I continued with the location factor, where the addresses in this column were recorded in a single cell with both the state's name and its abbreviation or sometime it could be a street name the state's abbreviation. Therefore, I split them into two separate columns and only took the abbreviation part for a bigger area of measurement.
After splitting the column and grouping the values, I noticed that there were 38 abbreviations and one value, "Anne Arundel." Therefore, I decided to create an additional column that referenced an external table containing the full names of the states. This would help readers easily recognize the abbreviations, as they might not remember all the letters representing each state. Additionally, "Anne Arundel" would be replaced with "MD" as it was a smaller region within the Maryland state.

In [16]:
#Took all the abbreviation
df['Location'] = df['Location'].str.split(',').str[1]

#Revaluing and stripping
df['Location'] = df['Location'].apply(lambda x: 'MD' if 'Anne' and 'Arundel' in str(x) else x)
df['Location'] = df['Location'].str.strip()
df['Location'] = df['Location'].str.upper()
#Load the file with full and abbr names
name_df = pd.read_excel(r'state_mapping.xlsx')

#Change cols' names for merging
df = df.rename(columns={'Location': 'Abbr Location'})
name_df = name_df.rename(columns={'2-letter USPS': 'Abbr Location'})
name_df['Abbr Location'] = name_df['Abbr Location'].str.strip()
name_df['Abbr Location'] = name_df['Abbr Location'].str.upper()

#merge df and name_df
merge_df = pd.merge(df,name_df,on='Abbr Location',how='left')
merge_df = merge_df.rename(columns={'Full Name': 'Full Location Name'})
full_location_name = merge_df.pop('Full Location Name') 
merge_df.insert(df.columns.get_loc('Abbr Location'), 'Full Location Name', full_location_name) 

After checking again using groupby, I found that the two tables had been completely merged, resulting in 39 grouped values as initially expected. Finally, I wanted to use the values from the "Rating" column to see which industry had the highest rating. I noticed that the rating values ranged from 0 to 5, so I wanted to drop if there were any values outside this range.

In [17]:
condition_4 = merge_df.index[merge_df['Full Location Name'].isnull()].tolist()
condition_5 = merge_df.index[merge_df['Abbr Location'].isnull()].tolist()
condition_6 = merge_df.index[df['Rating'] < 0].tolist()
condition_7 = merge_df.index[df['Rating'] > 5].tolist()

rating_and_remove_blanks_in_ful_and_abbr_col_condition = list(set(condition_4) | set(condition_5) | set(condition_6) | set(condition_7))
for x in rating_and_remove_blanks_in_ful_and_abbr_col_condition:
    merge_df = merge_df.drop(index=x)
    
#create a final file for visualization
merge_df.to_excel('DA_field_Jobs.xlsx', index=False)

merge_df

,Job Title,Salary Estimate,Min Sal,Max Sal,Job Description,Rating,Company Name,Full Location Name,Abbr Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$137K-$171K,137000,171000,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,New York,NY,"New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,$137K-$171K,137000,171000,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,Virginia,VA,"Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,$137K-$171K,137000,171000,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,Massachusetts,MA,"Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$137K-$171K,137000,171000,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,Massachusetts,MA,"Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,$137K-$171K,137000,171000,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,New York,NY,"New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,Data Scientist,$105K-$167K,105000,167000,Criterion Systems seeks a Data Scientist to su...,3.8,"Criterion Systems, Inc.\n3.8",Virginia,VA,"Vienna, VA",201 to 500 employees,2005,Company - Private,IT Services,Information Technology,$50 to $100 million (USD),-1
590,Data Scientist,$105K-$167K,105000,167000,About Foundation Medicine:\n\nFoundation Medic...,4.0,Foundation Medicine\n4.0,Massachusetts,MA,"Cambridge, MA",1001 to 5000 employees,2010,Company - Public,Biotech & Pharmaceuticals,Biotech & Pharmaceuticals,$100 to $500 million (USD),"Genomic Health, Myriad Genetics, The Broad Ins..."
591,Data Scientist,$105K-$167K,105000,167000,Summary\n\nWe’re looking for a data scientist ...,3.6,TRANZACT\n3.6,New Jersey,NJ,"Fort Lee, NJ",1001 to 5000 employees,1989,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,-1
592,Data Scientist,$105K-$167K,105000,167000,100 Remote Opportunity As an AINLP Data Scient...,5.0,ChaTeck Incorporated\n5.0,California,CA,"Santa Clara, CA",1 to 50 employees,-1,Company - Private,Advertising & Marketing,Business Services,$1 to $5 million (USD),-1


### Final Step

First, we looked at the impact of job titles on salaries.

In [18]:
merge_df.groupby('Job Title')[['Min Sal','Max Sal']].mean().round(0).sort_values(by='Max Sal',ascending=False).astype(int)


,Min Sal,Max Sal
Job Title,,
Data Architect,128000,201000
Data Scientist,99988,150062
Data Analyst,92611,139296
Machine Engineer,95169,137183
Data Modeler,98500,126250
Analytics Manager,104750,122250


Next, we looked at the impact of company size on salaries.

Looking at the table below, we could also conclude that company size does not determine salary levels in the data analytics field.

In [19]:
merge_df.groupby('Size')[['Min Sal','Max Sal']].mean().round(0).sort_values(by='Max Sal',ascending=False).astype(int)

,Min Sal,Max Sal
Size,,
51 to 200 employees,103390,151203
5001 to 10000 employees,104000,149750
10000+ employees,97507,147693
1001 to 5000 employees,97621,146252
1 to 50 employees,97379,145621
501 to 1000 employees,94133,143933
201 to 500 employees,95818,143831


Next, let's examine the top 10 highest-paying regions in the U.S.

In [20]:
merge_df.groupby(['Full Location Name','Abbr Location'])[['Min Sal','Max Sal']].mean().round(0).sort_values(by='Max Sal',ascending=False).astype(int).head(10)

,,Min Sal,Max Sal
Full Location Name,Abbr Location,,
Delaware,DE,212000,331000
District of Columbia,DC,121095,181333
North Carolina,NC,121222,179000
Iowa,IA,108333,173333
Wisconsin,WI,115667,173167
Arizona,AZ,115250,166500
Mississippi,MS,101000,165000
New Hampshire,NH,99500,164500
Rhode Island,RI,100500,161000


Next, let's look at the top 10 industries that would have the best salaries.

In [21]:
merge_df.groupby('Industry')[['Min Sal','Max Sal']].mean().round(0).sort_values(by='Max Sal',ascending=False).astype(int).head(10)

,Min Sal,Max Sal
Industry,,
"Health, Beauty, & Fitness",158500,249000
Transportation Equipment Manufacturing,141000,225000
State & Regional Agencies,141000,225000
"Hotels, Motels, & Resorts",141000,225000
Food & Beverage Stores,141000,225000
Wholesale,136333,217667
Other Retail Stores,129000,205667
Architectural & Engineering Services,118667,191000
Electrical & Electronic Manufacturing,122667,168333


After exploring the data, I found that the area in which an employee works, the industry they are in, and their job title have certain impacts on salary levels, while company size appears to be independent.